In [2]:
!pip install gensim transformers torch scikit-learn tqdm nltk
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from gensim.models import Doc2Vec
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from huggingface_hub import hf_hub_download
from google.colab import files

# Upload the file
uploaded = files.upload()

# Load the uploaded CSV into a DataFrame
df = pd.read_csv(next(iter(uploaded)))

# Map 'RequirementType' to 'labels' (Functional: 1, Non-Functional: 0)
label_mapping = {'F': 1, 'NF': 0}
df['labels'] = df['RequirementType'].map(label_mapping)


Saving final_corrected_fine_labeled_reviews.csv to final_corrected_fine_labeled_reviews (1).csv


In [3]:
# Tokenize documents into sentences
nltk.download('punkt')
df['sentences'] = df['content'].apply(sent_tokenize)

# Download and load the Doc2Vec model from Hugging Face
model_path = hf_hub_download(repo_id="RafidMehda/doc2vec_model", filename="doc2vec_model")
doc2vec_model = Doc2Vec.load(model_path)

# Load pre-trained DistilBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("RafidMehda/fined-distilBERT")
distilbert_model = AutoModel.from_pretrained("RafidMehda/fined-distilBERT")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Function to get Doc2Vec embeddings
def get_doc2vec_embeddings(index):
    doc2vec_emb = doc2vec_model.dv[str(index)]
    return doc2vec_emb

# Apply the function to extract Doc2Vec embeddings
doc2vec_embeddings = [get_doc2vec_embeddings(i) for i in range(len(df))]

# Function to get DistilBERT embeddings for sentences in a document
def get_distilbert_sentence_embeddings(sentences):
    sentence_embeddings = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)
        with torch.no_grad():
            outputs = distilbert_model(**inputs)
            last_hidden_state = outputs.last_hidden_state
            pooled_embedding = torch.mean(last_hidden_state, dim=1)
        sentence_embeddings.append(pooled_embedding.squeeze().numpy())
    return sentence_embeddings

# Apply the function to get DistilBERT embeddings for each document's sentences
df['sentence_embeddings'] = df['sentences'].apply(get_distilbert_sentence_embeddings)


In [5]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attn_weights = nn.Parameter(torch.Tensor(hidden_dim, 1))
        nn.init.xavier_uniform_(self.attn_weights)

    def forward(self, hidden_states):
        attn_scores = torch.tanh(torch.matmul(hidden_states, self.attn_weights)).squeeze(-1)
        attn_weights = F.softmax(attn_scores, dim=1)
        weighted_sum = torch.bmm(attn_weights.unsqueeze(1), hidden_states).squeeze(1)
        return weighted_sum, attn_weights

# Initialize the attention model
sentence_attention = Attention(hidden_dim=768)

# Apply attention mechanism
sentence_attention_outputs = []
for emb_list in df['sentence_embeddings']:
    sentence_embs = torch.tensor(np.array(emb_list))
    if len(sentence_embs.shape) == 2:
        sentence_embs = sentence_embs.unsqueeze(0)
    weighted_sum, _ = sentence_attention(sentence_embs)
    sentence_attention_outputs.append(weighted_sum.detach().numpy())


In [6]:
combined_embeddings = []
for doc2vec_emb, sentence_attn_emb in zip(doc2vec_embeddings, sentence_attention_outputs):
    sentence_attn_emb_flat = sentence_attn_emb.flatten()
    combined_embedding = np.concatenate((doc2vec_emb, sentence_attn_emb_flat))
    combined_embeddings.append(combined_embedding)

X = np.array(combined_embeddings)
y = df['labels'].values

# Split data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [11]:
# Define an adjusted Bi-LSTM model with two layers and reduced dropout
class AdjustedBiLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.1):
        super(AdjustedBiLSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=2, bidirectional=True, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, 0, :]  # Only use the first time step
        logits = self.fc(lstm_out)
        return self.softmax(logits)

# Initialize model with weight decay for L2 regularization and no dropout
input_dim = X_train.shape[1]
hidden_dim = 128
output_dim = 2
model = AdjustedBiLSTMClassifier(input_dim, hidden_dim, output_dim, dropout_prob=0.1)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # L2 regularization

# Training loop with early stopping
best_val_accuracy = 0
patience = 5
wait = 0
epochs = 50
for epoch in range(epochs):
    model.train()
    inputs = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # Add sequence dimension
    labels = torch.tensor(y_train, dtype=torch.long)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # Validation step
    model.eval()
    with torch.no_grad():
        val_inputs = torch.tensor(X_val, dtype=torch.float32).unsqueeze(1)
        val_labels = torch.tensor(y_val, dtype=torch.long)
        val_outputs = model(val_inputs)
        _, val_predicted = torch.max(val_outputs, 1)
        val_accuracy = accuracy_score(val_labels.numpy(), val_predicted.numpy())

    # Early stopping check
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        wait = 0
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        wait += 1
        if wait >= patience:
            print(f"Early stopping at epoch {epoch + 1}")
            break

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}, Validation Accuracy: {val_accuracy * 100:.2f}%")

# Load the best model for final evaluation
model.load_state_dict(torch.load('best_model.pt'))

# Evaluate the model on the test set
evaluate(X_train, y_train, "Training Set")
evaluate(X_val, y_val, "Validation Set")
evaluate(X_test, y_test, "Test Set")


Epoch 1/50, Loss: 0.6848, Validation Accuracy: 70.97%
Epoch 2/50, Loss: 0.6341, Validation Accuracy: 71.61%
Epoch 3/50, Loss: 0.5867, Validation Accuracy: 72.73%
Epoch 4/50, Loss: 0.5423, Validation Accuracy: 74.33%
Epoch 5/50, Loss: 0.5007, Validation Accuracy: 77.32%
Epoch 6/50, Loss: 0.4607, Validation Accuracy: 79.88%
Epoch 7/50, Loss: 0.4178, Validation Accuracy: 83.67%
Epoch 8/50, Loss: 0.3719, Validation Accuracy: 86.18%
Epoch 9/50, Loss: 0.3348, Validation Accuracy: 87.94%
Epoch 10/50, Loss: 0.3230, Validation Accuracy: 87.73%
Epoch 11/50, Loss: 0.3143, Validation Accuracy: 88.10%
Epoch 12/50, Loss: 0.2970, Validation Accuracy: 88.05%
Epoch 13/50, Loss: 0.2903, Validation Accuracy: 87.89%
Epoch 14/50, Loss: 0.2931, Validation Accuracy: 88.37%
Epoch 15/50, Loss: 0.2895, Validation Accuracy: 88.63%
Epoch 16/50, Loss: 0.2779, Validation Accuracy: 89.27%
Epoch 17/50, Loss: 0.2679, Validation Accuracy: 89.17%
Epoch 18/50, Loss: 0.2652, Validation Accuracy: 89.49%
Epoch 19/50, Loss: 

<ipython-input-11-fa105e2332ed>:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pt'))



Training Set Classification Report:
                precision    recall  f1-score   support

Non-Functional       0.97      0.95      0.96      4862
    Functional       0.94      0.96      0.95      3884

      accuracy                           0.96      8746
     macro avg       0.96      0.96      0.96      8746
  weighted avg       0.96      0.96      0.96      8746

Training Set Accuracy: 95.88%

Validation Set Classification Report:
                precision    recall  f1-score   support

Non-Functional       0.97      0.94      0.96      1045
    Functional       0.93      0.97      0.95       829

      accuracy                           0.95      1874
     macro avg       0.95      0.95      0.95      1874
  weighted avg       0.95      0.95      0.95      1874

Validation Set Accuracy: 95.30%

Test Set Classification Report:
                precision    recall  f1-score   support

Non-Functional       0.97      0.93      0.95      1036
    Functional       0.92      0.96   